In [52]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import string
import io
from google.colab import files
import gc

In [53]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
df_legit=pd.read_csv('drive/MyDrive/Legit_Data.csv')
df_phish=pd.read_csv('drive/MyDrive/Phish_Data.csv')

In [56]:
df_legit = df_legit.append(df_phish)
df_final=df_legit

In [57]:
df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]
df_final.reset_index(inplace=True)
df_final['index'] = df_final.index
df_final.tail()

,index,Subject,Body,Target
4580,4580,Congratulations!You're an eBay Silver PowerSe...,X-UID: 1335\n\nDear\n\n ...,1.0
4581,4581,Regular Account Maintenance,"X-UID: 13\n\nPayPal\n\n#yiv1429338740 ,#yiv14...",1.0
4582,4582,RE:Update our records!!,"BODY, TD\n{font-family:verdana,arial,helvetica...",1.0
4583,4583,Security Measures,X-UID: 1027\n\nPayPal\n#obmessage .dummy {}\n#...,1.0
4584,4584,***SPAM*** instructions from customer service...,X-UID: 1080\n\n This email sent to:\nuser@doma...,1.0


In [58]:
df_target=df_final.Target
df_target=df_target.astype(int)
del df_final['Target']
df_target.head()

0    0
1    0
2    0
3    0
4    0
Name: Target, dtype: int64

In [59]:
df_final['Subject']= df_final['Subject'].astype(str)
df_final['Body']= df_final['Body'].astype(str)

In [60]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [61]:
oversample = RandomOverSampler(sampling_strategy=0.25)
X_over, y_over = oversample.fit_resample(df_final, df_target)

In [62]:
y_over.value_counts()

0    4082
1    1020
Name: Target, dtype: int64

In [63]:
undersample = RandomUnderSampler(sampling_strategy=0.5)
X_under, y_under = undersample.fit_resample(X_over, y_over)

In [64]:
y_under.value_counts()

0    2040
1    1020
Name: Target, dtype: int64

In [65]:
y_under

0       0
1       0
2       0
3       0
4       0
       ..
3055    1
3056    1
3057    1
3058    1
3059    1
Name: Target, Length: 3060, dtype: int64

In [66]:
del X_under['index']
X_under.tail()

,Subject,Body
3055,JOB OFFER!!!!!!!!!!!!!!!!!!!!!!,X-UID: 32\n\nI am writing you from ukcorporato...
3056,Notification: Update Your eBay Profile,X-UID: 1114\n\nYour account will be suspended ...
3057,PayPal - Please Restore Your Account,X-UID: 125\n\nDear valued Citizens=AE Bank mem...
3058,"collection issue,","X-UID: 88\n\nDear \n User, \n\nWe n..."
3059,"Notification From: ""support""<user@domain>",Status: O\nContent-Length: 781\nLines: 16\n\nY...


In [67]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
from tqdm import tqdm

In [69]:
#Loading Glove Embedding
embeddings_index = {}
f = open('drive/MyDrive/glove.6B.50d.txt', encoding='utf8')
for line in tqdm(f):
    value = line.split()
    word = value[0]
    coef = np.asarray(value[1:], dtype='float32')
    embeddings_index[word] = coef
f.close()
print(len(embeddings_index))

400000it [00:06, 65884.73it/s]

400000


In [70]:
VOCABULARY_SIZE = 2000
MAX_LENGTH_BODY = 60
MAX_LENGTH_SUBJECT = 20

In [71]:
train_x, test_x, train_y, test_y = train_test_split(X_under, y_under, test_size=0.2, random_state=0, stratify=y_under)

In [72]:
print(len(test_x))

612


In [73]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [74]:
# tokenizing and padding email body
tokenizer_body = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer_body.fit_on_texts(list(train_x.Body) + list(test_x.Body))

xtrain_body_sequence = tokenizer_body.texts_to_sequences(train_x.Body)
xtest_body_sequence = tokenizer_body.texts_to_sequences(test_x.Body)

xtrain_body_padding = sequence.pad_sequences(xtrain_body_sequence, maxlen=MAX_LENGTH_BODY)
xtest_body_padding = sequence.pad_sequences(xtest_body_sequence, maxlen=MAX_LENGTH_BODY)
word_index_body = tokenizer_body.word_index

embedding_matrix_body = np.zeros((len(word_index_body) + 1, 50))
for word, i in tqdm(word_index_body.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_body[i] = embedding_vector

100%|██████████| 48607/48607 [00:00<00:00, 738920.26it/s]


In [75]:
# tokenizing and padding email subject
tokenizer_Subject = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer_Subject.fit_on_texts(list(train_x.Subject) + list(test_x.Subject))

xtrain_Subject_sequence = tokenizer_Subject.texts_to_sequences(train_x.Subject)
xtest_Subject_sequence = tokenizer_Subject.texts_to_sequences(test_x.Subject)

xtrain_Subject_padding = sequence.pad_sequences(xtrain_Subject_sequence, maxlen=MAX_LENGTH_SUBJECT)
xtest_Subject_padding = sequence.pad_sequences(xtest_Subject_sequence, maxlen=MAX_LENGTH_SUBJECT)
word_index_Subject = tokenizer_Subject.word_index

embedding_matrix_Subject = np.zeros((len(word_index_Subject) + 1, 50))
for word, i in tqdm(word_index_Subject.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_Subject[i] = embedding_vector

100%|██████████| 3151/3151 [00:00<00:00, 333096.05it/s]


In [76]:
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.layers.core import Dense, Activation, Dropout

In [77]:
from keras.models import Sequential

In [78]:
from keras.layers.embeddings import Embedding
import keras

In [79]:
#import packages
from time import time
from collections import Counter
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, KMeansSMOTE
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
'''stop_words = stopwords.words('english')'''
from sklearn.metrics import precision_recall_fscore_support

In [80]:
from keras.layers import Conv2D, MaxPooling2D, Input, Dense, Flatten, concatenate
from keras.models import Model

In [81]:
#LSTM
max_features=VOCABULARY_SIZE

In [82]:
xtrain_body_padding.shape

(2448, 60)

In [83]:
# creating the modelling branch for email body
body_input = Input(shape=(60,))

b1=Embedding(len(word_index_body) + 1,50,weights=[embedding_matrix_body],input_length=MAX_LENGTH_BODY,trainable=False)(body_input)
b2=SpatialDropout1D(0.3)(b1)
b3=Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3))(b2)
b4=Dense(1024, activation='relu')(b3)
b5=Dropout(0.4)(b4)
b6=Dense(1024, activation='relu')(b5)
b7=Dropout(0.4)(b6)

In [84]:
# creating the modelling branch for email Subject
Subject_input = Input(shape=(20,))

s1=Embedding(len(word_index_Subject) + 1,50,weights=[embedding_matrix_Subject],input_length=MAX_LENGTH_SUBJECT,trainable=False)(Subject_input)
s2=SpatialDropout1D(0.3)(s1)
s3=Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3))(s2)
s4=Dense(1024, activation='relu')(s3)
s5=Dropout(0.4)(b4)
s6=Dense(1024, activation='relu')(s5)
s7=Dropout(0.4)(s6)

In [85]:
# concatenating outputs from subject and body
concatenated = keras.layers.Add()([b7, s7])
c1=Dense(1024, activation='relu')(concatenated)
c2=Dropout(0.4)(c1)
c3=Dense(1)(c2)
out=Activation('sigmoid')(c3)

In [86]:
# compiling the model
model = Model([body_input, Subject_input], out)
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[keras.metrics.AUC()])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 60, 50)       2430400     ['input_3[0][0]']                
                                                                                                  
 spatial_dropout1d_2 (SpatialDr  (None, 60, 50)      0           ['embedding_2[0][0]']            
 opout1D)                                                                                         
                                                                                                  
 bidirectional_2 (Bidirectional  (None, 200)         120800      ['spatial_dropout1d_2[0][0]

In [87]:
early_stop = EarlyStopping(monitor='val_auc_1', mode='max', verbose=1, patience=3)
history = model.fit([xtrain_body_padding,xtrain_Subject_padding], train_y, batch_size=64, epochs=5, verbose=1, validation_data=([xtest_body_padding,xtest_Subject_padding],test_y),callbacks=[early_stop])

Epoch 1/5
39/39 [==============================] - 30s 614ms/step - loss: 0.4599 - auc_1: 0.8407 - val_loss: 0.2561 - val_auc_1: 0.9570
Epoch 2/5
39/39 [==============================] - 23s 588ms/step - loss: 0.2963 - auc_1: 0.9379 - val_loss: 0.2044 - val_auc_1: 0.9703
Epoch 3/5
39/39 [==============================] - 23s 592ms/step - loss: 0.2251 - auc_1: 0.9624 - val_loss: 0.1895 - val_auc_1: 0.9798
Epoch 4/5
39/39 [==============================] - 23s 592ms/step - loss: 0.1961 - auc_1: 0.9715 - val_loss: 0.1922 - val_auc_1: 0.9781
Epoch 5/5
39/39 [==============================] - 23s 587ms/step - loss: 0.1873 - auc_1: 0.9742 - val_loss: 0.1780 - val_auc_1: 0.9850


In [88]:
df_test=pd.read_csv('drive/MyDrive/Test Data.csv')

In [89]:
df_test.tail()

,Subject,Body
4893,RE: Highlights noname - let's catch up next week,Haha- no worries. I made some tweaks around t...
4894,Declined: Strategic Morning Huddle,Out on PTO
4895,RE: SV Initiative Page Approval,This makes sense to me. From: Diane M ...
4896,FW: [EXT]Re: ccac sales and walk NDA,Some ideas for use cases for 8/9 meeting ...
4897,RE: [EXT]Re: [EXT]Canceled: ccac sales: noname...,Perfect. Thank you! From: Mohdhafi Ag...


In [90]:
df_test['Subject']= df_test['Subject'].astype(str)
df_test['Body']= df_test['Body'].astype(str)

In [91]:
# tokenizing and padding pred emails body
xpred_body_sequence = tokenizer_body.texts_to_sequences(df_test.Body)
xpred_body_padding = sequence.pad_sequences(xpred_body_sequence, maxlen=MAX_LENGTH_BODY)

# tokenizing and padding pred emails subject
xpred_Subject_sequence = tokenizer_Subject.texts_to_sequences(df_test.Subject)
xpred_Subject_padding = sequence.pad_sequences(xpred_Subject_sequence, maxlen=MAX_LENGTH_SUBJECT)

In [92]:
predictions=model.predict([xpred_body_padding,xpred_Subject_padding])

In [93]:
predictions

array([[0.0558733 ],
       [0.0170344 ],
       [0.3159189 ],
       ...,
       [0.44541258],
       [0.04784732],
       [0.06915626]], dtype=float32)

In [94]:
pred_df=pd.DataFrame(predictions)
pred_df.tail()

,0
4893,0.009835
4894,0.003165
4895,0.445413
4896,0.047847
4897,0.069156


In [95]:
pred_df.to_csv('predictions_5.csv')